In [1]:
#Cert Active with DS_Customer
import pandas as pd

cert_active_df = pd.read_csv("Cert_AllActive.csv")
ds_customers_df = pd.read_csv("DS_Customers.csv")

/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_10700/515489967.py:4: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  cert_active_df = pd.read_csv("Cert_AllActive.csv")
/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_10700/515489967.py:5: DtypeWarning: Columns (13,48,56,64,69,84) have mixed types. Specify dtype option on import or set low_memory=False.
  ds_customers_df = pd.read_csv("DS_Customers.csv")


In [2]:
#Cert Active with DS_Customer
import pandas as pd

columns_ds_customers = [
    'Customer_ID', 'cert_type', 'cert_start_date', 'cert_end_date',
    'Stadard_Job_Title', 'job_function', 'job_title_freetext', 'companyname',
    'email', 'DISALLOW_ALL_COMMUNICATION', 'disallow_email_communication',
    'disallow_phone_communication', 'disallow_regular_mail_communi',
    'disallow_third_party_communic', 'membership_paid_through', 'employment_status',
    'department_size', 'gender', 'ethnicity', 'education', 'company_size',
    'SHRM_ENTERED_PROFESSION', 'shrm_birth_year', 'native_language', 
    'organization_unit', 'organization_type', 'employee_oversee', 'supervisor_title',
    'shrm_organization_multination', 'shrm_organization_unionized', 'SHRM_Join_Date',
    'Subsidiary', 'billaddr_1', 'billaddr_2', 'billaddr_3', 'billaddr_city',
    'billaddr_state', 'billaddr_zip', 'billaddr_country', 'billaddr_country_desc',
    'Current_Start_Date', 'Is_Person', 'IsInActive', 'LoginAccess', 'Monthly_Donor',
    'Industry_Category', 'SSO_UUID', 'Customer_Segment', 'Cohort'
]

columns_cert_active = [
    'ContactId', 'cobalt_name', 'Address1_Country', 'cobalt_expirationdate',
    'cobalt_enddate', 'Status Reason', 'Address1_Line1', 'Address1_City',
    'Address1_PostalCode', 'Address1_StateOrProvince', 'Address1_Composite',
    'EMailAddress1', 'BirthDate', 'shrm_startdate', 'shrm_InitialCertificationDate',
    'shrmadm_certificationbegindate', 'shrmadm_certificationenddate', 'shrm_SHRMMember',
    'shrm_currentbusinessindustry', 'shrm_currentbusinessindustryvalue',
    'shrm_currentpositiontitle', 'shrm_JobPosition', 'shrm_JobPositionvalue',
    'shrm_TimesRecertified', 'shrm_TimesRecertified_Date', 'shrm_TimesRecertified_State',
    'shrm_ssouid', 'SHRM_SSOEMail'
]

ds_customers_df = ds_customers_df[columns_ds_customers]
cert_active_df = cert_active_df[columns_cert_active]

# Perform the left join
result_df = pd.merge(ds_customers_df, cert_active_df,
                     left_on='email', right_on='EMailAddress1', how='left')

result_df['targetCertActive'] = result_df['EMailAddress1'].notna().replace({True: 'Yes', False: 'No'})

removal_df = pd.read_excel("/Users/muzammil.mahmud/Desktop/VSCode/Certification_Dataset/PropensityModel/DS_Transactions_Removal_IDs.xlsx")

#splitting based on yes and no values
df_no = result_df[result_df['targetCertActive'] == 'No']
df_yes = result_df[result_df['targetCertActive'] == 'Yes']

#merging back after removing customer ids that did not exist in DS_Transactions - we could keep this as training, and for testing we could include the ones that were removed because
#they did not exist in DS_Transactions - '''IMPORTANTTTTT!!!!!!!!!!!!!!!!!'''
reduced_df_no = pd.merge(df_no, removal_df,
                     left_on='email', right_on='Email',  # Simplified as the column name is the same in both DataFrames
                     how='inner')  # Changed from 'left' to 'inner' to ensure only overlapping IDs are included
final_df = pd.concat([reduced_df_no, df_yes], ignore_index=True)

final_df.columns

Index(['Customer_ID', 'cert_type', 'cert_start_date', 'cert_end_date',
       'Stadard_Job_Title', 'job_function', 'job_title_freetext',
       'companyname', 'email', 'DISALLOW_ALL_COMMUNICATION',
       'disallow_email_communication', 'disallow_phone_communication',
       'disallow_regular_mail_communi', 'disallow_third_party_communic',
       'membership_paid_through', 'employment_status', 'department_size',
       'gender', 'ethnicity', 'education', 'company_size',
       'SHRM_ENTERED_PROFESSION', 'shrm_birth_year', 'native_language',
       'organization_unit', 'organization_type', 'employee_oversee',
       'supervisor_title', 'shrm_organization_multination',
       'shrm_organization_unionized', 'SHRM_Join_Date', 'Subsidiary',
       'billaddr_1', 'billaddr_2', 'billaddr_3', 'billaddr_city',
       'billaddr_state', 'billaddr_zip', 'billaddr_country',
       'billaddr_country_desc', 'Current_Start_Date', 'Is_Person',
       'IsInActive', 'LoginAccess', 'Monthly_Donor', 'Indust

In [3]:
# Check the percentage of missing values in each column
missing_data = final_df.isnull().mean() * 100
print("Percentage of missing values in each column:")
print(missing_data)

Percentage of missing values in each column:
Customer_ID                     0.000000
cert_type                      82.009620
cert_start_date                82.007844
cert_end_date                  82.007844
Stadard_Job_Title              13.165988
                                 ...    
shrm_TimesRecertified_State    81.947756
shrm_ssouid                    81.951170
SHRM_SSOEMail                  81.947756
targetCertActive                0.000000
Email                          18.052244
Length: 79, dtype: float64


In [4]:
yes_count = final_df['targetCertActive'].value_counts().get('Yes', 0)
print(yes_count)

132189


In [5]:
#cert type imputation
#add place holder value for null

# Impute missing seniority_level with a placeholder
final_df['cert_type'].fillna('No_Type', inplace=True)

/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_40415/4286885342.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['cert_type'].fillna('No_Type', inplace=True)


In [6]:
#cert_start_date, cert_end_date columns feature engineering

final_df['cert_start_date'] = pd.to_datetime(final_df['cert_start_date'])

# Reference date for calculating days since the event (example: today)
reference_date = pd.to_datetime('2024-01-01')

# Days since the event for those who attended
final_df['days_since_cert_start_date'] = (reference_date - final_df['cert_start_date']).dt.days

# Fill missing values for non-attendees with a large constant
final_df['days_since_cert_start_date'].fillna(36500, inplace=True)

# Additional temporal features (optional)
final_df['cert_start_date_month'] = final_df['cert_start_date'].dt.month.fillna(0).astype(int)  # 0 for non-attendees
final_df['cert_start_date_weekday'] = final_df['cert_start_date'].dt.weekday.fillna(0).astype(int)  # 0 for non-attendees


final_df['cert_end_date'] = pd.to_datetime(final_df['cert_end_date'])
# Reference date for calculating days since the event (example: today)
reference_date = pd.to_datetime('2024-01-01')

# Days since the event for those who attended
final_df['days_since_cert_start_date'] = (reference_date - final_df['cert_start_date']).dt.days

# Fill missing values for non-attendees with a large constant
final_df['days_since_cert_start_date'].fillna(36500, inplace=True)

# Additional temporal features (optional)
final_df['cert_start_date_month'] = final_df['cert_start_date'].dt.month.fillna(0).astype(int)  # 0 for non-attendees
final_df['cert_start_date_weekday'] = final_df['cert_start_date'].dt.weekday.fillna(0).astype(int)  # 0 for non-attendees

/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_40415/478340932.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  final_df['cert_start_date'] = pd.to_datetime(final_df['cert_start_date'])
/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_40415/478340932.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['days_since_cert_start_date'].fillna(36500, inplace=Tr

In [7]:
#Initial Imputation for Stadard Job Title
import pandas as pd
from sklearn.impute import SimpleImputer, KNNImputer

# Function to fill missing Stadard_Job_Title using job_function and shrm_JobPositionvalue
def fill_job_title(row):
    if pd.isna(row['Stadard_Job_Title']):
        if pd.notna(row['job_function']):
            return row['job_function']
        elif pd.notna(row['shrm_JobPositionvalue']):
            return row['shrm_JobPositionvalue']
    return row['Stadard_Job_Title']

final_df['Stadard_Job_Title'] = final_df.apply(fill_job_title, axis=1)
print("Missing values in 'Stadard_Job_Title' after custom imputation:", final_df['Stadard_Job_Title'].isnull().sum())

#Simple Imputation step 2
# If there are still missing values, use a simple imputer (e.g., most frequent)
imputer = SimpleImputer(strategy='most_frequent')
# Note the addition of [:, 0] to correctly shape the array for the DataFrame column
final_df['Stadard_Job_Title'] = imputer.fit_transform(final_df[['Stadard_Job_Title']])[:, 0]

# Verify all missing values are filled
print("Missing values in 'Stadard_Job_Title' after full imputation:", final_df['Stadard_Job_Title'].isnull().sum())

Missing values in 'Stadard_Job_Title' after custom imputation: 85735
Missing values in 'Stadard_Job_Title' after full imputation: 0


In [8]:
# Intelligent imputation for 'companyname'
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Assuming 'final_df' is your DataFrame
# First, fill other categorical variables with a placeholder to stabilize the base
categorical_vars = ['company_size', 'organization_type', 'department_size', 'organization_unit']

# Use SimpleImputer to fill missing values with the most frequent category
imputer = SimpleImputer(strategy='most_frequent') #--could be replaced with new category
final_df[categorical_vars] = imputer.fit_transform(final_df[categorical_vars])

# Encoding categorical variables
le = LabelEncoder()
for column in categorical_vars:
    final_df[column] = le.fit_transform(final_df[column])

# Preparing the feature columns and the target column
features = final_df[categorical_vars]
target = final_df['companyname']

# Encode the target 'companyname'
target_filled = target.dropna()
le_company = LabelEncoder()
target_filled_encoded = le_company.fit_transform(target_filled)

# Splitting data that has 'companyname' available into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features.loc[target_filled.index], target_filled_encoded, test_size=0.2, random_state=42)

# Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predicting missing 'companyname'
missing_indexes = final_df[target.isnull()].index
predicted_names_encoded = clf.predict(features.loc[missing_indexes])
predicted_names = le_company.inverse_transform(predicted_names_encoded)

# Filling in the missing 'companyname' in the original DataFrame
final_df.loc[missing_indexes, 'companyname'] = predicted_names

# Calculate and print the percentage distribution of the imputed company names
imputed_distribution = pd.Series(predicted_names).value_counts(normalize=True) * 100
print("Distribution of imputed 'company names' as percentages:")
print(imputed_distribution)

Distribution of imputed 'company names' as percentages:
The Boeing Company             67.001549
Retired                         4.098323
Self Employed                   3.346546
ADP                             1.701750
Paychex/ Inc.                   1.272325
                                 ...    
Air Force Aid Society Inc       0.001139
Benhaven Inc                    0.001139
ABM Industries Incorporated     0.001139
APEX BENEFITS GROUP/ INC.       0.001139
AFB International               0.001139
Name: proportion, Length: 1327, dtype: float64


In [9]:
#Emplyee Oversee Imputation
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Fill missing values in categorical predictors
imputer = SimpleImputer(strategy='most_frequent')
final_df[categorical_vars] = imputer.fit_transform(final_df[categorical_vars])

# Assuming categorical_vars are previously defined and include relevant predictors
features = final_df[categorical_vars]
features = pd.get_dummies(features)  # Convert categorical variables to dummy/indicator variables

# Encode and split the target variable 'employee_oversee'
target = final_df['employee_oversee'].dropna()
le = LabelEncoder()
target_encoded = le.fit_transform(target)

# Only select rows where 'employee_oversee' is not missing
known_indexes = final_df[~final_df['employee_oversee'].isnull()].index
X_train, X_test, y_train, y_test = train_test_split(features.loc[known_indexes], target_encoded, test_size=0.2, random_state=42)

# Train a Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict the missing values
missing_indexes = final_df[final_df['employee_oversee'].isnull()].index
if len(missing_indexes) > 0:
    predicted_values_encoded = clf.predict(features.loc[missing_indexes])
    predicted_values = le.inverse_transform(predicted_values_encoded)

    # Fill in the missing 'employee_oversee' in the original DataFrame
    final_df.loc[missing_indexes, 'employee_oversee'] = predicted_values

    # Output distribution
    imputed_distribution = pd.Series(predicted_values).value_counts(normalize=True) * 100
    print("Distribution of imputed 'employee_oversee' as percentages:")
    print(imputed_distribution)
else:
    print("No missing 'employee_oversee' to impute.")

Distribution of imputed 'employee_oversee' as percentages:
0              95.718204
2-4             3.803001
10-24           0.160466
5-9             0.157601
25-49           0.137543
1               0.012243
100-499         0.009378
50-99           0.001302
500 or more     0.000260
Name: proportion, dtype: float64


In [10]:
#Supervisor Title Imputation
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# Fill missing values in categorical predictors
imputer = SimpleImputer(strategy='most_frequent')
final_df[categorical_vars] = imputer.fit_transform(final_df[categorical_vars])

# Assuming categorical_vars are previously defined and include relevant predictors
features = final_df[categorical_vars]
features = pd.get_dummies(features)  # Convert categorical variables to dummy/indicator variables

# Encode and split the target variable 'employee_oversee'
target = final_df['supervisor_title'].dropna()
le = LabelEncoder()
target_encoded = le.fit_transform(target)

# Only select rows where 'employee_oversee' is not missing
known_indexes = final_df[~final_df['supervisor_title'].isnull()].index
X_train, X_test, y_train, y_test = train_test_split(features.loc[known_indexes], target_encoded, test_size=0.2, random_state=42)

# Train a Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict the missing values
missing_indexes = final_df[final_df['supervisor_title'].isnull()].index
if len(missing_indexes) > 0:
    predicted_values_encoded = clf.predict(features.loc[missing_indexes])
    predicted_values = le.inverse_transform(predicted_values_encoded)

    # Fill in the missing 'employee_oversee' in the original DataFrame
    final_df.loc[missing_indexes, 'supervisor_title'] = predicted_values

    # Output distribution
    imputed_distribution = pd.Series(predicted_values).value_counts(normalize=True) * 100
    print("Distribution of imputed 'supervisor_title' as percentages:")
    print(imputed_distribution)
else:
    print("No missing 'supervisor_title' to impute.")

Distribution of imputed 'supervisor_title' as percentages:
President/ CEO/ Chairman                  75.264072
Director of HR                            23.535050
HR Manager                                 0.948117
Other                                      0.154494
I'm Highest Ranked In My Org               0.051409
Administrator                              0.020617
VP of HR                                   0.014191
Chief of Staff/ Div Head/ Dept Chair       0.002410
Partner/ Principal/ Executive Director     0.002142
Dean/ Superintendent/ General Manager      0.002142
CHRO/ CHCO                                 0.001874
Exec Officer (Govt/Military)               0.001339
CFO                                        0.001071
SVP of HR or other SVP                     0.000536
COO                                        0.000536
Name: proportion, dtype: float64


In [11]:
#Imputation for gender, ethnicity, educationimport pandas as pd

# Assuming 'final_df' is your DataFrame containing the data

# Replace missing values with 'Unknown' in specified columns
final_df['gender'] = final_df['gender'].fillna('Undisclosed')
final_df['ethnicity'] = final_df['ethnicity'].fillna('Undisclosed')
final_df['education'] = final_df['education'].fillna('Not Identified')
final_df['native_language'] = final_df['native_language'].fillna('Undisclosed')

# Check the changes
print("Updated counts for 'gender':")
print(final_df['gender'].value_counts())

print("\nUpdated counts for 'ethnicity':")
print(final_df['ethnicity'].value_counts())

print("\nUpdated counts for 'education':")
print(final_df['education'].value_counts())

print("\nUpdated counts for 'native_language':")
print(final_df['native_language'].value_counts())


Updated counts for 'gender':
gender
Female         369572
Undisclosed    279413
Male            83273
Name: count, dtype: int64

Updated counts for 'ethnicity':
ethnicity
Undisclosed                         318187
White                               281480
Black/African American               51520
Hispanic/White Latino                20605
Asian                                19266
Hispanic/Other Latino                12929
Hispanic                              9999
Multicultural/Other                   8977
Other                                 3678
American Indian/Alaskan Native        3120
Native Hawaiian/Pacific Islander      1316
Asian/Pacific-Islander                1181
Name: count, dtype: int64

Updated counts for 'education':
education
Not Identified               297805
Bachelor's Degree/ non-HR    112208
Bachelor's Degree/ HR         53244
Master's Degree/ HR           48607
Bachelor's Degree             34086
Master's Degree/ non-HR       33647
High School / GED           

In [12]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

join_date_mode = final_df['SHRM_Join_Date'].dropna().mode()[0]
final_df['SHRM_Join_Date'] = pd.to_datetime(final_df['SHRM_Join_Date']).fillna(join_date_mode)

# Convert SHRM_Join_Date to an integer: number of days from a specific date (e.g., the mode date)
final_df['SHRM_Join_Date_int'] = (final_df['SHRM_Join_Date'] - datetime(2000, 1, 1)).dt.days

# Encode the 'standard_job_title' column
encoder_job_title = LabelEncoder()
final_df['Stadard_Job_Title_encoded'] = encoder_job_title.fit_transform(final_df['Stadard_Job_Title'].astype(str))

# Prepare the missing column, assume 'SHRM_ENTERED_PROFESSION' is categorical
final_df['SHRM_ENTERED_PROFESSION'] = final_df['SHRM_ENTERED_PROFESSION'].astype(str)  # Ensure data type is consistent
encoder_profession = LabelEncoder()
final_df['SHRM_ENTERED_PROFESSION_encoded'] = encoder_profession.fit_transform(final_df['SHRM_ENTERED_PROFESSION'])

# Separate the data into sets with and without missing values
is_missing = final_df['SHRM_ENTERED_PROFESSION'] == 'nan'
train_df = final_df[~is_missing]  # non-missing data
test_df = final_df[is_missing]    # missing data

# Train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(train_df[['Stadard_Job_Title_encoded', 'SHRM_Join_Date_int']], train_df['SHRM_ENTERED_PROFESSION_encoded'])

# Predict the missing values
predicted = model.predict(test_df[['Stadard_Job_Title_encoded', 'SHRM_Join_Date_int']])

# Inverse transform the predictions back to original labels
test_df['SHRM_ENTERED_PROFESSION'] = encoder_profession.inverse_transform(predicted)

# Replace the missing values in the original dataframe
final_df.loc[is_missing, 'SHRM_ENTERED_PROFESSION'] = test_df['SHRM_ENTERED_PROFESSION']

# Optionally, verify and print some of the imputed values
print("Sample of imputed SHRM_ENTERED_PROFESSION values:")
print(final_df.loc[is_missing, 'SHRM_ENTERED_PROFESSION'].head())

/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_40415/1244724403.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  final_df['SHRM_Join_Date'] = pd.to_datetime(final_df['SHRM_Join_Date']).fillna(join_date_mode)


Sample of imputed SHRM_ENTERED_PROFESSION values:
8     2015.0
14    2015.0
18    2012.0
19    2012.0
20    2019.0
Name: SHRM_ENTERED_PROFESSION, dtype: object


/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_40415/1244724403.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['SHRM_ENTERED_PROFESSION'] = encoder_profession.inverse_transform(predicted)


In [13]:
import pandas as pd

# Set the threshold for dropping columns, in this case, 80%
threshold = 80.0

# Calculate the percentage of missing values for each column
missing_percentages = final_df.isnull().mean() * 100

# Find columns that have missing data above the threshold
columns_to_drop = missing_percentages[missing_percentages > threshold].index

# Drop these columns from the DataFrame
final_df.drop(columns=columns_to_drop, inplace=True)

# Print the remaining columns to verify
print("Remaining columns after dropping:")
print(final_df.columns)
print("\nDropped columns:")
print(columns_to_drop)

Remaining columns after dropping:
Index(['Customer_ID', 'cert_type', 'Stadard_Job_Title', 'job_function',
       'job_title_freetext', 'companyname', 'email',
       'DISALLOW_ALL_COMMUNICATION', 'disallow_email_communication',
       'disallow_phone_communication', 'disallow_regular_mail_communi',
       'disallow_third_party_communic', 'membership_paid_through',
       'department_size', 'gender', 'ethnicity', 'education', 'company_size',
       'SHRM_ENTERED_PROFESSION', 'shrm_birth_year', 'native_language',
       'organization_unit', 'organization_type', 'employee_oversee',
       'supervisor_title', 'shrm_organization_multination',
       'shrm_organization_unionized', 'SHRM_Join_Date', 'Subsidiary',
       'billaddr_1', 'billaddr_city', 'billaddr_state', 'billaddr_zip',
       'billaddr_country', 'billaddr_country_desc', 'Current_Start_Date',
       'Is_Person', 'IsInActive', 'LoginAccess', 'Monthly_Donor',
       'Industry_Category', 'SSO_UUID', 'Customer_Segment', 'targetCertA

In [14]:
#'cert_type', 'Current_Start_Date','days_since_cert_start_date', 'cert_start_date_month','cert_start_date_weekday'

In [15]:
import pandas as pd

# Convert 'membership_paid_through' to datetime
final_df['membership_paid_through'] = pd.to_datetime(final_df['membership_paid_through'])

# Reference date for calculating days since the membership paid through date (example: '2024-01-01')
reference_date = pd.to_datetime('2024-05-24')

# Calculate days since the membership was paid through
final_df['days_since_membership_paid_through'] = (final_df['membership_paid_through']-reference_date).dt.days

# Fill missing values for days_since_membership_paid_through with a large constant (assuming non-payment or long time since payment)
final_df['days_since_membership_paid_through'].fillna(36500, inplace=True)

# Additional temporal features
final_df['membership_paid_through_month'] = final_df['membership_paid_through'].dt.month.fillna(0).astype(int)  # 0 for missing dates
final_df['membership_paid_through_weekday'] = final_df['membership_paid_through'].dt.weekday.fillna(0).astype(int)  # 0 for missing dates

# Print sample to verify the changes
print(final_df[['membership_paid_through', 'days_since_membership_paid_through', 'membership_paid_through_month', 'membership_paid_through_weekday']].head())

  membership_paid_through  days_since_membership_paid_through  \
0              2024-04-30                               -24.0   
1              2019-08-31                             -1728.0   
2              2024-12-31                               221.0   
3              2020-03-31                             -1515.0   
4              2024-11-30                               190.0   

   membership_paid_through_month  membership_paid_through_weekday  
0                              4                                1  
1                              8                                5  
2                             12                                1  
3                              3                                1  
4                             11                                5  


/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_40415/3916047560.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  final_df['membership_paid_through'] = pd.to_datetime(final_df['membership_paid_through'])
/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_40415/3916047560.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['days_since_membership_paid_through'

In [16]:
import pandas as pd

# Fill null values in 'Industry_Category' with 'unknown'
final_df['Industry_Category'] = final_df['Industry_Category'].fillna('unknown')

# Verify the operation by checking if there are any null values left in 'Industry_Category'
print("Null values in 'Industry_Category' after filling:", final_df['Industry_Category'].isnull().sum())

Null values in 'Industry_Category' after filling: 0


In [17]:
import pandas as pd

# Assuming 'final_df' is your DataFrame

# Convert 'Current_Start_Date' to datetime
final_df['Current_Start_Date'] = pd.to_datetime(final_df['Current_Start_Date'])

# Reference date for calculating days since the event (example: today)
reference_date = pd.to_datetime('2024-01-01')

# Days since the event for those who have a start date
final_df['days_since_current_start_date'] = (reference_date - final_df['Current_Start_Date']).dt.days

# Fill missing values with a placeholder date
final_df['Current_Start_Date'].fillna(pd.to_datetime('1900-01-01'), inplace=True)  # Change '1900-01-01' as needed

# Additional temporal features (optional)
final_df['current_start_date_month'] = final_df['Current_Start_Date'].dt.month.fillna(0).astype(int)  # 0 for missing values
final_df['current_start_date_weekday'] = final_df['Current_Start_Date'].dt.weekday.fillna(0).astype(int)  # 0 for missing values

/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_40415/2928127789.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  final_df['Current_Start_Date'] = pd.to_datetime(final_df['Current_Start_Date'])
/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_40415/2928127789.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Current_Start_Date'].fillna(pd.to_datetime('1

In [18]:
import pandas as pd

# Assuming 'final_df' is your DataFrame
# Load your DataFrame (uncomment and modify the path if starting from a CSV file)
# final_df = pd.read_csv('path_to_your_file.csv')

# List of columns to be dropped
columns_to_drop = ['job_function', 'job_title_freetext', 'Email', 'SSO_UUID', 'membership_paid_through']

# Check and drop only if the column is present in the DataFrame
columns_to_drop = [col for col in columns_to_drop if col in final_df.columns]
final_df.drop(columns=columns_to_drop, inplace=True)

# Print the remaining columns to verify
print("Remaining columns after dropping:")
print(final_df.columns)

Remaining columns after dropping:
Index(['Customer_ID', 'cert_type', 'Stadard_Job_Title', 'companyname', 'email',
       'DISALLOW_ALL_COMMUNICATION', 'disallow_email_communication',
       'disallow_phone_communication', 'disallow_regular_mail_communi',
       'disallow_third_party_communic', 'department_size', 'gender',
       'ethnicity', 'education', 'company_size', 'SHRM_ENTERED_PROFESSION',
       'shrm_birth_year', 'native_language', 'organization_unit',
       'organization_type', 'employee_oversee', 'supervisor_title',
       'shrm_organization_multination', 'shrm_organization_unionized',
       'SHRM_Join_Date', 'Subsidiary', 'billaddr_1', 'billaddr_city',
       'billaddr_state', 'billaddr_zip', 'billaddr_country',
       'billaddr_country_desc', 'Current_Start_Date', 'Is_Person',
       'IsInActive', 'LoginAccess', 'Monthly_Donor', 'Industry_Category',
       'Customer_Segment', 'targetCertActive', 'days_since_cert_start_date',
       'cert_start_date_month', 'cert_start_da

In [19]:
import pandas as pd

# Assuming 'final_df' is your DataFrame
# Load your DataFrame (uncomment and modify the path if starting from a CSV file)
# final_df = pd.read_csv('path_to_your_file.csv')

# List of columns to fill with the mode
columns_to_fill = [
    'billaddr_1', 'billaddr_city', 'billaddr_state', 'billaddr_zip', 
    'billaddr_country', 'billaddr_country_desc', 'shrm_organization_multination',
    'shrm_organization_unionized', 'shrm_birth_year', 'disallow_phone_communication',
    'disallow_third_party_communic', 'Monthly_Donor'
]

# Loop through each column and fill missing values with the mode
for column in columns_to_fill:
    # Calculate the mode of the column
    mode_value = final_df[column].mode()[0]  # mode() returns a Series; use [0] to get the mode value
    
    # Fill missing values with the mode
    final_df[column].fillna(mode_value, inplace=True)
    
    # Optionally print the mode used for each column
    print(f"Filled missing values in {column} with mode: {mode_value}")

# Check if any of these columns still have missing values
print("\nMissing values after filling:")
for column in columns_to_fill:
    print(f"{column}: {final_df[column].isnull().sum()} missing values")

/var/folders/p6/lz9nv0md22z21j6w4rb19b380000gp/T/ipykernel_40415/96620848.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[column].fillna(mode_value, inplace=True)


Filled missing values in billaddr_1 with mode: 1800 Duke St
Filled missing values in billaddr_city with mode: New York
Filled missing values in billaddr_state with mode: CA
Filled missing values in billaddr_zip with mode: 00000
Filled missing values in billaddr_country with mode: US
Filled missing values in billaddr_country_desc with mode: United States
Filled missing values in shrm_organization_multination with mode: F
Filled missing values in shrm_organization_unionized with mode: F
Filled missing values in shrm_birth_year with mode: 0.0
Filled missing values in disallow_phone_communication with mode: F
Filled missing values in disallow_third_party_communic with mode: F
Filled missing values in Monthly_Donor with mode: F

Missing values after filling:
billaddr_1: 0 missing values
billaddr_city: 0 missing values
billaddr_state: 0 missing values
billaddr_zip: 0 missing values
billaddr_country: 0 missing values
billaddr_country_desc: 0 missing values
shrm_organization_multination: 0 mis

In [20]:
import pandas as pd

# Assuming 'final_df' is your DataFrame
# Set display options to avoid truncation
pd.set_option('display.max_rows', None)  # Adjust as necessary for row display
pd.set_option('display.max_columns', None)  # Adjust as necessary for column display
pd.set_option('display.width', None)  # Adjusts the width of the display to show entire columns
pd.set_option('display.max_colwidth', None)  # Ensure full content of each column is shown

# Check the percentage of missing values in each column
missing_data = final_df.isnull().mean() * 100
print("Percentage of missing values in each column:")
print(missing_data)

Percentage of missing values in each column:
Customer_ID                           0.000000
cert_type                             0.000000
Stadard_Job_Title                     0.000000
companyname                           0.000000
email                                 0.000000
DISALLOW_ALL_COMMUNICATION            0.000000
disallow_email_communication          0.000000
disallow_phone_communication          0.000000
disallow_regular_mail_communi         0.000000
disallow_third_party_communic         0.000000
department_size                       0.000000
gender                                0.000000
ethnicity                             0.000000
education                             0.000000
company_size                          0.000000
SHRM_ENTERED_PROFESSION               0.000000
shrm_birth_year                       0.000000
native_language                       0.000000
organization_unit                     0.000000
organization_type                     0.000000
employee_overse

In [21]:
#Decide Imputation strategy
df = final_df
# Create a DataFrame to hold the summary
summary_df = pd.DataFrame(columns=['Column Name', 'Null Count', 'Distinct Count', 'Total Count', 'Data Type'])

# Iterate over each column in the original DataFrame
for column in df.columns:
    # Create a temporary DataFrame for current column statistics
    temp_df = pd.DataFrame({
        'Column Name': [column],
        'Null Count': [df[column].isnull().sum()],
        'Distinct Count': [df[column].nunique()],
        'Total Count': [df[column].size],
        'Data Type': [df[column].dtype]
    })
    # Concatenate the temporary DataFrame to the summary DataFrame
    summary_df = pd.concat([summary_df, temp_df], ignore_index=True)

# Adjust pandas display settings
pd.set_option('display.max_rows', None)  # Adjust to show all rows
pd.set_option('display.max_columns', None)  # Adjust to show all columns
pd.set_option('display.width', 1000)  # Adjust the width to fit your screen
pd.set_option('display.max_colwidth', None)  # Adjust to show full content of each column

# Display the summary DataFrame
print(summary_df)

                           Column Name Null Count Distinct Count Total Count       Data Type
0                          Customer_ID          0         474692      732258         float64
1                            cert_type          0              3      732258          object
2                    Stadard_Job_Title          0             47      732258          object
3                          companyname          0         381931      732258          object
4                                email          0         731091      732258          object
5           DISALLOW_ALL_COMMUNICATION          0              2      732258          object
6         disallow_email_communication          0              2      732258          object
7         disallow_phone_communication          0              2      732258          object
8        disallow_regular_mail_communi          0              2      732258          object
9        disallow_third_party_communic          0              2      

In [22]:
df = final_df
df = df.drop(['companyname', 'Stadard_Job_Title', 'SHRM_ENTERED_PROFESSION', 'cert_start_date_month', 'cert_start_date_weekday','Customer_ID', 'email', 'SHRM_Join_Date', 'Current_Start_Date', 'cert_type', 'days_since_current_start_date', 'billaddr_1', 'current_start_date_month', 'current_start_date_weekday','days_since_cert_start_date'], axis=1, errors='ignore')
df.columns

Index(['DISALLOW_ALL_COMMUNICATION', 'disallow_email_communication', 'disallow_phone_communication', 'disallow_regular_mail_communi', 'disallow_third_party_communic', 'department_size', 'gender', 'ethnicity', 'education', 'company_size', 'shrm_birth_year', 'native_language', 'organization_unit', 'organization_type', 'employee_oversee', 'supervisor_title', 'shrm_organization_multination', 'shrm_organization_unionized', 'Subsidiary', 'billaddr_city', 'billaddr_state', 'billaddr_zip', 'billaddr_country', 'billaddr_country_desc', 'Is_Person', 'IsInActive', 'LoginAccess', 'Monthly_Donor', 'Industry_Category', 'Customer_Segment', 'targetCertActive', 'SHRM_Join_Date_int', 'Stadard_Job_Title_encoded', 'SHRM_ENTERED_PROFESSION_encoded', 'days_since_membership_paid_through', 'membership_paid_through_month', 'membership_paid_through_weekday'], dtype='object')

In [23]:
df.columns

Index(['DISALLOW_ALL_COMMUNICATION', 'disallow_email_communication', 'disallow_phone_communication', 'disallow_regular_mail_communi', 'disallow_third_party_communic', 'department_size', 'gender', 'ethnicity', 'education', 'company_size', 'shrm_birth_year', 'native_language', 'organization_unit', 'organization_type', 'employee_oversee', 'supervisor_title', 'shrm_organization_multination', 'shrm_organization_unionized', 'Subsidiary', 'billaddr_city', 'billaddr_state', 'billaddr_zip', 'billaddr_country', 'billaddr_country_desc', 'Is_Person', 'IsInActive', 'LoginAccess', 'Monthly_Donor', 'Industry_Category', 'Customer_Segment', 'targetCertActive', 'SHRM_Join_Date_int', 'Stadard_Job_Title_encoded', 'SHRM_ENTERED_PROFESSION_encoded', 'days_since_membership_paid_through', 'membership_paid_through_month', 'membership_paid_through_weekday'], dtype='object')

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, recall_score
import category_encoders as ce

# Sample DataFrame
# df = pd.read_csv('your_dataset.csv')  # Ensure df is loaded with your dataset

# Encode the target variable
df['targetCertActive'] = df['targetCertActive'].map({'Yes': 1, 'No': 0})

# Initialize dictionaries to store mappings
binary_mappings = {}
one_hot_mappings = {}
target_mappings = {}

# Binary Encoding
binary_cols = [
    'DISALLOW_ALL_COMMUNICATION',
    'disallow_email_communication',
    'disallow_phone_communication',
    'disallow_regular_mail_communi',
    'disallow_third_party_communic',
    'shrm_organization_multination',
    'shrm_organization_unionized',
    'Subsidiary',
    'Is_Person',
    'IsInActive',
    'LoginAccess',
    'Monthly_Donor'
]

# Create mappings and encode binary columns
for col in binary_cols:
    mapping = {'Yes': 1, 'No': 0, True: 1, False: 0, pd.NA: -1, None: -1}
    binary_mappings[col] = mapping
    df[col] = df[col].map(mapping).fillna(-1)

print("Binary encoding complete")

# One-Hot Encoding
one_hot_cols = [
    #'Stadard_Job_Title',
    'gender',
    'ethnicity',
    'education',
   # 'SHRM_ENTERED_PROFESSION',
    'native_language',
    'employee_oversee',
    'supervisor_title',
    'Industry_Category',
    'Customer_Segment'
]

# Store original categories
for col in one_hot_cols:
    one_hot_mappings[col] = list(df[col].unique())

df = pd.get_dummies(df, columns=one_hot_cols)
print("One-Hot encoding complete")

# Target Encoding
target_encode_cols = [
    #'companyname',
    'billaddr_city',
    'billaddr_state',
    'billaddr_zip',
    'billaddr_country',
    'billaddr_country_desc'
]

# Store mappings for target encoding
target_encoder = ce.TargetEncoder(cols=target_encode_cols)
df[target_encode_cols] = target_encoder.fit_transform(df[target_encode_cols], df['targetCertActive'])

for col in target_encode_cols:
    target_mappings[col] = target_encoder.mapping[col]

print("Target encoding complete")

# Define features and target variable
X = df.drop('targetCertActive', axis=1)
y = df['targetCertActive']

Binary encoding complete
One-Hot encoding complete
Target encoding complete


In [25]:
# Assuming 'final_df' is your DataFrame
# Set display options to avoid truncation
pd.set_option('display.max_rows', None)  # Adjust as necessary for row display
pd.set_option('display.max_columns', None)  # Adjust as necessary for column display
pd.set_option('display.width', None)  # Adjusts the width of the display to show entire columns
pd.set_option('display.max_colwidth', None)  # Ensure full content of each column is shown

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define features and target variable
X = df.drop('targetCertActive', axis=1)
y = df['targetCertActive']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform k-fold cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5)

# Print cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Cross-Validation Score:", cv_scores.mean())

# Train the model on the full training set
model.fit(X_train, y_train)

# Make predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Evaluate the model
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)
classification_rep = classification_report(y_test, y_pred_test)
conf_matrix = confusion_matrix(y_test, y_pred_test)

# Print evaluation metrics
print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
print("Classification Report:")
print(classification_rep)
print("Confusion Matrix:")
print(conf_matrix)

Cross-Validation Scores: [0.93345965 0.93303232 0.93296404 0.93350176 0.93140209]
Mean Cross-Validation Score: 0.9328719736778378
Train Accuracy: 0.9990474662260203
Test Accuracy: 0.9325717641274957
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.98      0.96    120088
           1       0.88      0.73      0.80     26364

    accuracy                           0.93    146452
   macro avg       0.91      0.85      0.88    146452
weighted avg       0.93      0.93      0.93    146452

Confusion Matrix:
[[117358   2730]
 [  7145  19219]]


In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, recall_score, confusion_matrix

def evaluate_model(clf, X_test, y_test):
    """Evaluate the model and print metrics."""

    # Predict probabilities for the positive class
    propensity_scores = clf.predict_proba(X_test)[:, 1]

    # Generate binary predictions based on a default threshold of 0.5
    predictions = (propensity_scores > 0.5).astype(int)

    # Calculate AUC using the probability scores
    y_test = y_test.astype(int)
    auc_score = roc_auc_score(y_test, propensity_scores)

    # Calculate recall using the binary predictions
    recall = recall_score(y_test, predictions)

    # Print the basic evaluation metrics
    print(f"***** AUC = {auc_score:.2%}    Recall = {recall:.2%}  ***** \n")

    # Threshold analysis
    thresholds = [0.5, 0.4, 0.3, 0.2, 0.1, 0.01, 0.005, 0.001, 0.0001]
    auc_list, recall_list, target_list, confusion_matrices = [], [], [], []

    for threshold in thresholds:
        # Generate binary predictions based on the current threshold
        y_pred_threshold = (propensity_scores > threshold).astype(int)

        # Calculate metrics for the current threshold
        auc_list.append(roc_auc_score(y_test, y_pred_threshold))
        recall_list.append(recall_score(y_test, y_pred_threshold))
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_threshold).ravel()
        target_list.append((fp + tp) / len(y_test))
        confusion_matrices.append(f"[{tn},{fp}],[{fn},{tp}]")

    # Prepare and print the summary table
    summary_table = pd.DataFrame({
        'Threshold': thresholds,
        'AUC:': [f"{x:.2%}" for x in auc_list],
        'Recall:': [f"{x:.2%}" for x in recall_list],
        'Target %': [f"{x:.3%}" for x in target_list],
        'Confusion Matrix': confusion_matrices
    })
    print(summary_table)

def get_feature_importances(clf, features):
    """Get feature importances from the trained model."""
    importances = clf.feature_importances_
    sorted_idx = np.argsort(importances)[::-1]
    return np.array(features)[sorted_idx], importances[sorted_idx]

# Define features and target variable
X = df.drop('targetCertActive', axis=1)
y = df['targetCertActive']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the full training set
model.fit(X_train, y_train)

# Evaluate the model using the custom function
evaluate_model(model, X_test, y_test)

***** AUC = 97.13%    Recall = 72.90%  ***** 

   Threshold    AUC: Recall: Target %             Confusion Matrix
0     0.5000  85.31%  72.90%  14.987%   [117358,2730],[7145,19219]
1     0.4000  88.53%  81.08%  17.894%   [115257,4831],[4989,21375]
2     0.3000  90.74%  87.86%  21.044%   [112432,7656],[3200,23164]
3     0.2000  91.61%  92.91%  24.678%  [108442,11646],[1869,24495]
4     0.1000  89.25%  97.14%  32.779%    [97693,22395],[753,25611]
5     0.0100  70.08%  99.54%  66.604%    [48788,71300],[121,26243]
6     0.0050  64.18%  99.75%  76.489%     [34366,85722],[67,26297]
7     0.0010  64.15%  99.75%  76.544%     [34285,85803],[67,26297]
8     0.0001  64.15%  99.75%  76.548%     [34279,85809],[67,26297]
